# 03_feature_engineering
Placeholder notebook created by script.

Load Clean Data

In [8]:
import pandas as pd

df = pd.read_csv("../data/processed_sales.csv")
df['Order Date'] = pd.to_datetime(df['Order Date'])
df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,City,State,...,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,Shipping Cost,Order Priority
0,32298,CA-2012-124891,2012-07-31,2012-07-31,Same Day,RH-19495,Rick Hansen,Consumer,New York City,New York,...,TEC-AC-10003033,Technology,Accessories,Plantronics CS510 - Over-the-Head monaural Wir...,2309.650,7,0.0,762.1845,933.57,Critical
1,26341,IN-2013-77878,2013-02-05,2013-02-07,Second Class,JR-16210,Justin Ritter,Corporate,Wollongong,New South Wales,...,FUR-CH-10003950,Furniture,Chairs,"Novimex Executive Leather Armchair, Black",3709.395,9,0.1,-288.7650,923.63,Critical
2,25330,IN-2013-71249,2013-10-17,2013-10-18,First Class,CR-12730,Craig Reiter,Consumer,Brisbane,Queensland,...,TEC-PH-10004664,Technology,Phones,"Nokia Smart Phone, with Caller ID",5175.171,9,0.1,919.9710,915.49,Medium
3,13524,ES-2013-1579342,2013-01-28,2013-01-30,First Class,KM-16375,Katherine Murray,Home Office,Berlin,Berlin,...,TEC-PH-10004583,Technology,Phones,"Motorola Smart Phone, Cordless",2892.510,5,0.1,-96.5400,910.16,Medium
4,47221,SG-2013-4320,2013-11-05,2013-11-06,Same Day,RH-9495,Rick Hansen,Consumer,Dakar,Dakar,...,TEC-SHA-10000501,Technology,Copiers,"Sharp Wireless Fax, High-Speed",2832.960,8,0.0,311.5200,903.04,Critical


Create Time Features

In [9]:
df['Year'] = df['Order Date'].dt.year
df['Month'] = df['Order Date'].dt.month
df['Quarter'] = df['Order Date'].dt.quarter


Aggregate Monthly Sales

In [10]:
monthly_sales = (
    df.groupby(['Year', 'Month', 'Quarter'])['Sales']
      .sum()
      .reset_index()
)

monthly_sales.head()


,Year,Month,Quarter,Sales
0,2011,1,1,98898.48886
1,2011,2,1,91152.15698
2,2011,3,1,145729.36736
3,2011,4,2,116915.76418
4,2011,5,2,146747.83610


Check for Missing Months

In [11]:
monthly_sales.isnull().sum()
monthly_sales = monthly_sales.sort_values(['Year', 'Month'])
display(monthly_sales)

,Year,Month,Quarter,Sales
0,2011,1,1,98898.48886
1,2011,2,1,91152.15698
2,2011,3,1,145729.36736
3,2011,4,2,116915.76418
4,2011,5,2,146747.83610
5,2011,6,2,215207.38022
6,2011,7,3,115510.41912
7,2011,8,3,207581.49122
8,2011,9,3,290214.45534
9,2011,10,4,199071.26404


Lag Feature

In [12]:
monthly_sales['Prev_month_sales'] = monthly_sales['Sales'].shift(1)
monthly_sales.dropna(inplace=True)


Define Features & Target

In [13]:
X = monthly_sales[['Year', 'Month', 'Quarter', 'Prev_month_sales']]
y = monthly_sales['Sales']


Save ML-Ready Dataset

In [14]:
monthly_sales.to_csv("../data/ml_monthly_sales.csv", index=False)
